# RAG using Meta AI Llama-3.2


<img src="./resources/rag_architecture.png" width=800px>

In [2]:
import nest_asyncio
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, StorageContext

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client

In [3]:
# allows nested access to the event loop
nest_asyncio.apply()

In [4]:
# add your documents in this directory, you can drag & drop
input_dir_path = './docs'

In [7]:
collection_name="chat_with_docs"

client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

def create_index(documents):
    vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
    )
    return index

In [8]:

# setup llm & embedding model
llm=Ollama(model="llama3.2:1b", request_timeout=120.0)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

In [9]:
# load data
loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

# Creating an index over loaded data
Settings.embed_model = embed_model
try:
    index = create_index(docs)
    print('Using Qdrant collection')
except:
    index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine, where we use a cohere reranker on the fetched nodes
Settings.llm = llm
query_engine = index.as_query_engine()

# ====== Customise prompt template ======
qa_prompt_tmpl_str = (
"Context information is below.\n"
"---------------------\n"
"{context_str}\n"
"---------------------\n"
"Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
"Query: {query_str}\n"
"Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response
response = query_engine.query("What exactly is DSPy?",)

Using Qdrant collection


In [10]:
display(Markdown(str(response)))

To determine what DSPy is, we can follow these steps:

1. Understand the context provided.
2. Identify the key components mentioned in the text.
3. Consider how these components relate to each other.

From the provided information, we can infer that DSPy is a framework or library for natural language processing (NLP). Here's a step-by-step breakdown:

4. **Context**: The text mentions "document-chat-rag/docs/dspy.pdf", which suggests it's related to AI engineering and documentation chat applications.

5. **Key Components**:
   - Natural Language Signature: A way of defining tasks or functions that can be executed by a Large Model (LM).
   - Signatures are defined using a tuple of input fields and output fields, along with optional instructions.
   - DSPy provides a shorthand notation for these signatures, allowing them to be expressed in a more concise manner.

6. **Relationships**: The text highlights the advantages of using DSPy over traditional prompting techniques, such as being able to compile self-improving and pipeline-adaptive prompts.

7. **Scope**: It's implied that DSPy is designed to support various NLP tasks, including question answering, text transformation, and more.

8. **Building Blocks**: The text mentions several built-in modules, including Predict, ChainOfThought, ProgramOfThought, MultiChainComparison, and ReAct, which can be used interchangeably to implement DSPy signatures.

Given these points, it appears that DSPy is a framework or library for natural language processing that enables the creation of custom NLP tasks (signatures) using concise notation, with built-in modules providing various tools for implementing these tasks.